In [30]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse, urlunparse
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
tqdm.pandas()

## **Fonctions**

In [31]:
def fix_encoding(text):
    if isinstance(text, str):
        try:
            return text.encode("latin1").decode("utf-8")
        except:
            return text
    return text

In [32]:
def extract_domain(url):
    parsed = urlparse(url)
    domain = parsed.netloc.lower()

    # Retirer "www."
    if domain.startswith("www."):
        domain = domain[4:]
    
    return domain

In [33]:
def extract_domain_name(url):
    parsed = urlparse(url)
    domain = (parsed.hostname or "").lower()

    # enlever "www."
    domain = domain.removeprefix("www.")

    parts = domain.split(".")

    if len(parts) >= 2:
        return parts[-2]   # avant le dernier point
    else:
        return domain

In [34]:
def find_tag(url, list_tag):
    if not isinstance(url, str):
        return ""

    url = url.lower()

    for tag in list_tag:
        if tag in url:
            return tag
    
    return ""

In [35]:
def clean_url(url):
    """Supprime http, https, www et garde uniquement domaine + chemin."""
    url = url.strip().lower()

    # enlever http:// ou https://
    url = url.replace("https://", "").replace("http://", "")

    # enlever www.
    url = url.removeprefix("www.")

    # enlever trailing slash
    if url.endswith("/"):
        url = url[:-1]

    return url

def get_anchor_text(page_url, target_url):
    try:
        response = requests.get(page_url, timeout=10, headers={"User-Agent": "Mozilla/5.0"})
        response.raise_for_status()

    except Exception as e:
        return None, f"Page inaccessible: {e}"

    soup = BeautifulSoup(response.text, "lxml")

    # normalisation du lien à chercher
    target_clean = clean_url(target_url)

    # Parcourir tous les <a>
    for a in soup.find_all("a", href=True):
        href = a["href"].strip().lower()

        # cas 1 : href complet → nettoyage
        href_clean = clean_url(href)

        # cas 2 : href relatif → on compare sans domaine
        # exemple : target = "example.com/page"
        #           href = "/page"
        if href_clean in target_clean or target_clean in href_clean:
            anchor = a.get_text(strip=True)
            return anchor, None

    return None, "Lien introuvable dans la page"

## **Données**

In [36]:
df = pd.read_excel("Soumettre.xlsx")
for col in df.columns:
    df[col] = df[col].map(fix_encoding)

In [37]:
df

,URL,Tag,Plateforme,link_to_check,anchor_text
0,NaN,promovacances,Soumettre,https://www.promovacances.com/vacances-sejour-...,NaN
1,https://www.destinationlemonde.com/destination...,promovacances,Soumettre,https://www.promovacances.com/vacances-sejour-...,séjour en République Dominicaine tout compris
2,NaN,promovacances,Soumettre,https://www.promovacances.com/vacances-sejour-...,NaN
3,NaN,promovacances,Soumettre,https://www.promovacances.com/vacances-sejour-...,NaN
4,https://www.saddy.fr/voyages/avantages-des-cir...,promovacances,Soumettre,https://www.promovacances.com/vacances/circuit...,Découvrir le Kenya en circuit touristique
...,...,...,...,...,...
3119,https://www.chateau-conros.com/sur-quelle-ile-...,promovacances,Soumettre,https://www.promovacances.com/vacances-sejour-...,faire un voyage en Grèce
3120,https://www.terresdenvies.com/quelles-activite...,promovacances,Soumettre,https://www.promovacances.com/vacances-sejour-...,Pendant votre Voyage en Grèce
3121,https://www.quartdetours.fr/opter-pour-un-voya...,promovacances,Soumettre,https://www.promovacances.com/vacances-sejour-...,au cours de votre voyage en Grèce
3122,https://www.evalys-bus.fr/quels-sont-les-avant...,promovacances,Soumettre,https://www.promovacances.com/vacances-sejour-...,optez pour un voyage Grèce


## **1) Traitement des tags**

In [38]:
mask = df['link_to_check'].notna()
df.loc[mask, 'Tag'] = df.loc[mask, 'link_to_check'].astype(str).apply(extract_domain_name)

## **2) Traitement des ancres et des backlinks**

In [39]:
mask_search_anchor = df['URL'].notna() & df['anchor_text'].isna()

df[mask_search_anchor]

,URL,Tag,Plateforme,link_to_check,anchor_text
46,https://www.paris-normandie.fr/id657777/articl...,promovacances,Soumettre,https://www.promovacances.com/,NaN
68,https://www.courrier-picard.fr/id654609/articl...,promovacances,Soumettre,https://www.promovacances.com/vacances-sejour-...,NaN
69,https://www.lemessager.fr/649340692/article/20...,promovacances,Soumettre,https://www.promovacances.com/vacances-sejour-...,NaN
191,https://www.webwiki.fr/promocroisiere.com,,Soumettre,http://,NaN
207,https://www.lavoixdunord.fr/1625592/article/20...,promocroisiere,Soumettre,https://www.promocroisiere.com/fr/croisieres/c...,NaN
223,https://www.webwiki.fr/abcroisiere.com,,Soumettre,http://,NaN
239,https://www.lavoixdunord.fr/1614215/article/20...,abcroisiere,Soumettre,https://www.abcroisiere.com/fr/croisieres/croi...,NaN
270,https://voyages-intrepides.com/croatie-comment...,promovacances,Soumettre,https://www.promovacances.com/vacances-sejour-...,NaN
350,https://wholeterrain.org/divertissement/croisi...,promocroisiere,Soumettre,https://www.promocroisiere.com/fr/bateau-norwe...,NaN
363,https://www.fox-voyage.fr/croisiere-autour-du-...,promocroisiere,Soumettre,https://www.promocroisiere.com/fr/croisieres/c...,NaN


In [40]:
def process_row(row):
    page_url = str(row['URL'])
    target = str(row['link_to_check'])

    if page_url == 'nan' or target == 'nan':
        return pd.Series({"anchor_text": None, "error": "URL vide"})

    anchor, error = get_anchor_text(page_url, target)
    return pd.Series({"anchor_text": anchor, "error": error})

In [41]:
df.loc[mask_search_anchor, ['anchor_text', 'error']] = (
    df.loc[mask_search_anchor].progress_apply(process_row, axis=1)
)

100%|██████████| 47/47 [01:49<00:00,  2.33s/it]


In [43]:
df[mask_search_anchor]

,URL,Tag,Plateforme,link_to_check,anchor_text,error
46,https://www.paris-normandie.fr/id657777/articl...,promovacances,Soumettre,https://www.promovacances.com/,None,Page inaccessible: 403 Client Error: Forbidden...
68,https://www.courrier-picard.fr/id654609/articl...,promovacances,Soumettre,https://www.promovacances.com/vacances-sejour-...,None,Page inaccessible: 403 Client Error: Forbidden...
69,https://www.lemessager.fr/649340692/article/20...,promovacances,Soumettre,https://www.promovacances.com/vacances-sejour-...,None,Page inaccessible: 403 Client Error: Forbidden...
191,https://www.webwiki.fr/promocroisiere.com,,Soumettre,http://,,None
207,https://www.lavoixdunord.fr/1625592/article/20...,promocroisiere,Soumettre,https://www.promocroisiere.com/fr/croisieres/c...,None,Page inaccessible: 403 Client Error: Forbidden...
223,https://www.webwiki.fr/abcroisiere.com,,Soumettre,http://,,None
239,https://www.lavoixdunord.fr/1614215/article/20...,abcroisiere,Soumettre,https://www.abcroisiere.com/fr/croisieres/croi...,None,Page inaccessible: 403 Client Error: Forbidden...
270,https://voyages-intrepides.com/croatie-comment...,promovacances,Soumettre,https://www.promovacances.com/vacances-sejour-...,sur Promovacances,None
350,https://wholeterrain.org/divertissement/croisi...,promocroisiere,Soumettre,https://www.promocroisiere.com/fr/bateau-norwe...,,None
363,https://www.fox-voyage.fr/croisiere-autour-du-...,promocroisiere,Soumettre,https://www.promocroisiere.com/fr/croisieres/c...,une croisière autour du monde,None


In [47]:
df['error'].unique()

array([nan,
       'Page inaccessible: 403 Client Error: Forbidden for url: https://www.paris-normandie.fr/id657777/article/2025-09-10/voyage-ou-partir-pour-pas-cher-en-automne',
       'Page inaccessible: 403 Client Error: Forbidden for url: https://www.courrier-picard.fr/id654609/article/2025-08-27/voyage-aux-seychelles-comment-bien-choisir-son-hebergement',
       'Page inaccessible: 403 Client Error: Forbidden for url: https://www.lemessager.fr/649340692/article/2025-08-27/top-5-des-plus-belles-plages-de-l-ile-maurice',
       None,
       'Page inaccessible: 403 Client Error: Forbidden for url: https://www.lavoixdunord.fr/1625592/article/2025-09-17/msc-yacht-club-l-experience-vip-qui-vaut-le-coup',
       'Page inaccessible: 403 Client Error: Forbidden for url: https://www.lavoixdunord.fr/1614215/article/2025-08-11/msc-croisieres-pourquoi-cette-compagnie-seduit-les-familles-francaises',
       'Page inaccessible: 404 Client Error: Not Found for url: https://www.provenceweb.fr/f/ac

In [52]:
df.to_excel("Soumettre_vdef.xlsx", index=False, engine="openpyxl")